In [24]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
import sys
import importlib
from threading import Thread
import time # for time.sleep
from ipygany import PolyMesh, Scene, IsoColor, WarpByScalar
import pyvista as pv
# from pyvista import _vtk
import pyvistaqt as pvqt
import colorcet as cc # Colormaps:
import numpy as np
import h5py
import hdf5storage # conda install hdf5storage
from pathlib import Path
import bqplot.scales
import seaborn as sns
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
# import mplcursors
import math # For color map generation
from matplotlib.colors import ListedColormap
from matplotlib.cm import hsv

import ipywidgets as widgets
# from PyQt5 import QtWidgets, uic
from pyvistaqt import QtInteractor, MainWindow
# from pyqt6 import QApplication
from IPython.external.qt_for_kernel import QtGui
from PyQt5.QtWidgets import QApplication

import datetime as dt
import panel as pn # for interactive widgets
pn.extension()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
# import PhoPositionalData as pdp
# from PhoPositionalData import load_exported, process_data
from PhoPositionalData.load_exported import *
# from PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
from PhoPositionalData.process_data import *
from PhoPositionalData.plot_data import *
from PhoPositionalData.plotting.animations import * # make_mp4_from_plotter
from PhoPositionalData.import_data import * # build_spike_positions_list, build_cellID_reverse_lookup_map
from PhoPositionalData.analysis.interactive_placeCell_config import InteractivePlaceCellConfig, VideoOutputModeConfig, PlottingConfig, PlacefieldComputationParameters  # VideoOutputModeConfig, InteractivePlaceCellConfigs
from PhoPositionalData.analysis.interactive_placeCell_config import print_subsession_neuron_differences

In [26]:
# NeuroPy (Diba Lab Python Repo) Loading
try:
    from neuropy import core
    importlib.reload(core)
except ImportError:
    sys.path.append(r'C:\Users\Pho\repos\NeuroPy') # Windows
    # sys.path.append('/home/pho/repo/BapunAnalysis2021/NeuroPy') # Linux
    # sys.path.append(r'/Users/pho/repo/Python Projects/NeuroPy') # MacOS
    print('neuropy module not found, adding directory to sys.path. \n >> Updated sys.path.')
    from neuropy import core
from neuropy.core.dataSession import SessionConfig, DataSessionLoader, DataSession, processDataSession
from neuropy.core.epoch import NamedEpoch
from neuropy.core.neurons import Neurons

## Data must be pre-processed using the MATLAB script located here: 
# R:\data\KDIBA\gor01\one\IIDataMat_Export_ToPython_2021_11_23.m

# From pre-computed .mat files:
## 07: 
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53'

# ## 08:
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15'

In [27]:
## Bapun Format:
# basedir = '/media/share/data/Bapun/Day5TwoNovel' # Linux
basedir = Path('R:\data\Bapun\Day5TwoNovel') # Windows
# basedir = '/Volumes/iNeo/Data/Bapun/Day5TwoNovel' # MacOS
sess = DataSessionLoader.bapun_data_session(basedir)
active_sess_config = sess.config
session_name = sess.name


converting neuron_type strings to core.neurons.NeuronType objects...
	 done.
linearized position loaded from file.
Loading success: .flattened.spikes.npy.
Loading success: .ripple.npy.
Loading success: .mua.npy.
Loading success: .pbe.npy.


In [28]:
sess.epochs.to_dataframe()


,start,stop,label,duration
0,0,11066,pre,11066
1,11070,13970,maze1,2900
3,13972,20754,post1,6782
2,20756,24004,maze2,3248
4,24006,42305,post2,18299


In [29]:
sess.flattened_spiketrains.spikes_df

,flat_spike_idx,t_seconds,aclu,unit_id,cell_type,x,y,linear_pos,speed,t
0,0,0.045933,51,51,NeuronType.INTERNEURONS,-50.946354,-50.646282,NaN,0.0,4.593333e+02
1,1,0.047367,88,88,NeuronType.CONTAMINATED,-50.946354,-50.646282,NaN,0.0,4.736667e+02
2,2,0.047700,86,86,NeuronType.INTERNEURONS,-50.946354,-50.646282,NaN,0.0,4.770000e+02
3,3,0.083400,88,88,NeuronType.CONTAMINATED,-50.946354,-50.646282,NaN,0.0,8.340000e+02
4,4,0.087167,15,15,NeuronType.CONTAMINATED,-50.946354,-50.646282,NaN,0.0,8.716667e+02
...,...,...,...,...,...,...,...,...,...,...
16318812,16318812,42305.809900,100,100,NeuronType.CONTAMINATED,-45.350155,64.150757,NaN,0.0,4.230581e+08
16318813,16318813,42305.810533,100,100,NeuronType.CONTAMINATED,-45.350155,64.150757,NaN,0.0,4.230581e+08
16318814,16318814,42305.810567,66,66,NeuronType.CONTAMINATED,-45.350155,64.150757,NaN,0.0,4.230581e+08
16318815,16318815,42305.814767,32,32,NeuronType.CONTAMINATED,-45.350155,64.150757,NaN,0.0,4.230581e+08


In [38]:
## NOTE: The only place that I need to be careful with indexing is with sess.position properties, as these appear to be represented in terms of the number of 60Hz samples instead of in seconds like the Neurons and other classes.
active_epoch_name = 'maze2'
# active_epoch_name = 'maze2'

active_epoch = NamedEpoch(
        name=active_epoch_name,
        start_end_times=sess.epochs[active_epoch_name]
    )

# active_epoch_name = 'maze'
# active_epoch = NamedEpoch(
#         name=active_epoch_name,
#         start_end_times=[sess.epochs['maze1'][0], sess.epochs['maze2'][1]]
#     )
# Constraining to epoch with times (start: 0.0, end: 1932.4200048116618)
# 20996/932598 total spikes spanning 40/64 units remain in subsession

active_subplots_shape = (1,1) # Single subplot
# active_subplots_shape = '1|2' # 1 subplot on left, two on right

## Get the config corresponding to this epoch/session settings:
# active_config = InteractivePlaceCellConfig(active_session_config=active_sess_config, active_epochs=active_epoch)
active_config = InteractivePlaceCellConfig(active_session_config=active_sess_config, active_epochs=active_epoch,
                        video_output_config=VideoOutputModeConfig(active_frame_range=np.arange(11070.0, 13970.0), video_output_parent_dir=Path('output', active_sess_config.session_name, active_epoch.name), active_is_video_output_mode=False),
                        plotting_config=PlottingConfig(output_subplots_shape=active_subplots_shape, output_parent_dir=Path('output', active_sess_config.session_name, active_epoch.name))) # '3|1    
## Looks like the only real outputs are: active_epoch_session_Neurons, active_epoch_pos
active_config.plotting_config.active_output_parent_dir.mkdir(parents=True, exist_ok=True) # makes the directory if it isn't already there

# Get the filtered and time-constrainted session and position data for this epoch:
active_epoch_session = sess.get_neuron_type('pyramidal').time_slice(active_epoch.t_start, active_epoch.t_stop)
print_subsession_neuron_differences(sess.neurons, active_epoch_session.neurons)
active_epoch_session_Neurons, active_epoch_pos, active_epoch_position_times = active_epoch_session.neurons, active_epoch_session.position, active_epoch_session.position.time

Constraining to units with type: pyramidal
Constraining to epoch with times (start: 20756, end: 24004)
105840/16318817 total spikes spanning 52/109 units remain in subsession


In [39]:
## Compute Placefields if needed:
from neuropy.analyses import Pf1D, Pf2D, perform_compute_placefields, plot_all_placefields
from neuropy.plotting.spikes import get_neuron_colors

should_force_recompute_placefields = True
should_display_2D_plots = False
# Setup the placefield computation config
active_config.computation_config = PlacefieldComputationParameters(speed_thresh=3, grid_bin=2, smooth=2)
# print_subsession_neuron_differences(sess.neurons, active_epoch_session_Neurons)

try: active_epoch_placefields1D
except NameError: active_epoch_placefields1D = None # Checks variable active_epoch_placefields's existance, and sets its value to None if it doesn't exist so it can be checked in the next step

try: active_epoch_placefields2D
except NameError: active_epoch_placefields2D = None # Checks variable active_epoch_placefields's existance, and sets its value to None if it doesn't exist so it can be checked in the next step

active_epoch_placefields1D, active_epoch_placefields2D = perform_compute_placefields(active_epoch_session_Neurons, active_epoch_pos, active_config.computation_config, active_epoch_placefields1D, active_epoch_placefields2D, should_force_recompute_placefields=True)
# Plot the placefields computed and save them out to files:
if should_display_2D_plots:
    ax_pf_1D, occupancy_fig, active_pf_2D_figures = plot_all_placefields(active_epoch_placefields1D, active_epoch_placefields2D, active_config)
else:
    print('skipping 2D placefield plots')
    
# Focus on the 2D placefields:
active_epoch_placefields = active_epoch_placefields2D
# Get the cell IDs that have a good place field mapping:
good_placefield_neuronIDs = np.array(active_epoch_placefields.ratemap.neuron_ids) # in order of ascending ID
print('good_placefield_neuronIDs: {}; ({} good)'.format(good_placefield_neuronIDs, len(good_placefield_neuronIDs)))

## Filter by neurons with good placefields only:
active_epoch_session = active_epoch_session.get_by_id(good_placefield_neuronIDs) # Filter by good placefields only, and this fetch also ensures they're returned in the order of sorted ascending index ([ 2  3  5  7  9 12 18 21 22 23 26 27 29 34 38 45 48 53 57])
active_epoch_session_Neurons = active_epoch_session.neurons

## Plotting Colors:
pf_sort_ind = np.array([int(i) for i in np.arange(len(active_epoch_session.neuron_ids))]) # convert to integer scalar array
pf_colors = get_neuron_colors(pf_sort_ind) # colors are by ascending index ID
active_config.plotting_config.pf_sort_ind = pf_sort_ind
active_config.plotting_config.pf_colors = pf_colors
active_config.plotting_config.active_cells_colormap = pf_colors.T # Make the colormap from the listed colors, used seemingly only by 'runAnalysis_PCAandICA(...)'
active_config.plotting_config.active_cells_listed_colormap = ListedColormap(active_config.plotting_config.active_cells_colormap)

Recomputing active_epoch_placefields...
	 done.
Recomputing active_epoch_placefields2D...
	 done.
skipping 2D placefield plots
good_placefield_neuronIDs: [ 1  3  8 11 13 20 27 34 37 38 41 42 43 44 47 48 49 53 54 55 56 57 58 61
 62 63 64 65 68 70 75 77 80 87 91 93 95]; (37 good)


In [40]:
## ICA and PCA Analysis
should_show_2D_ICA_plots = False
from PhoPositionalData.analysis.neuronal_dimensionality_reduction import runAnalysis_PCAandICA
active_session_ensembles, template, zsc_template, pca_data = runAnalysis_PCAandICA(active_epoch_session_Neurons, bin_size=0.250, frate_thresh=0.1, should_plot=should_show_2D_ICA_plots, active_cells_colormap=active_config.plotting_config.active_cells_colormap)

Based on frate_thresh, excluded neuron_ids: [ 8 34 37 41 49 80]


## Main Spike/Placemap plotting:

# Plot Placemaps on 3D Plot

In [41]:
from PhoGui.InteractivePlotter.InteractivePlaceCellTuningCurvesDataExplorer import InteractivePlaceCellTuningCurvesDataExplorer

t = active_epoch_pos.time
x = active_epoch_pos.x
y = active_epoch_pos.y

## Slider with Callback Function Example:
from PhoGui.InteractivePlotter.InteractivePlaceCellTuningCurvesDataExplorer import InteractivePlaceCellTuningCurvesDataExplorer

try: pActiveTuningCurvesPlotter
except NameError: pActiveTuningCurvesPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
ipcDataExplorer = InteractivePlaceCellTuningCurvesDataExplorer(active_config, x, y, active_epoch_placefields, pf_colors, extant_plotter=pActiveTuningCurvesPlotter)

In [42]:
pActiveTuningCurvesPlotter = ipcDataExplorer.plot(pActiveTuningCurvesPlotter)

No open BackgroundPlotter
Creating a new BackgroundPlotter
done.


C:\Users\Pho\repos\NeuroPy\neuropy\utils\mathutil.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (x - np.min(x, axis=axis, keepdims=True)) / np.ptp(


num_curr_tuning_curves: 37


In [43]:
from PhoGui.InteractivePlotter.shared_helpers import InteractivePyvistaPlotterBuildIfNeededMixin
from PhoGui.InteractivePlotter.InteractivePlaceCellDataExplorer import InteractivePlaceCellDataExplorer

active_config.plotting_config.show_legend = True

try: pActiveInteractivePlaceSpikesPlotter
except NameError: pActiveInteractivePlaceSpikesPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
ipspikesDataExplorer = InteractivePlaceCellDataExplorer(active_config, active_epoch_session, t, x, y, extant_plotter=pActiveInteractivePlaceSpikesPlotter)
print('done.')

active_epoch_pos.sampling_rate (Hz): 60
longer_spikes_window - curr_view_window_length_samples - 61440
recent_spikes_window - curr_view_window_length_samples - 60
Applying custom Pyvista theme.
done.
done.


In [44]:
pActiveInteractivePlaceSpikesPlotter = ipspikesDataExplorer.plot(pActivePlotter=pActiveInteractivePlaceSpikesPlotter)

No extant BackgroundPlotter
Creating a new BackgroundPlotter
done.
[f] - Focus and zoom in on the last clicked point
shift+click - Drag to pan the rendering scene
ctrl+click - Rotate the scene in 2D
